# ARTchives: a data driven historiography of art history - Jupyter Notebook

This Jupyter Notebook is integral part of the research carried out by Lucia Giagnolini for her master's thesis in Knowledge organization and Cultural Heritage, Digital Humanities and Digital Knowledge international degree a. y. 2020/2021.

The focus of the thesis is the recently published ARTchives project, which can be accessed at: http://artchives.fondazionezeri.unibo.it. ARTchives is the first international web-based platform for a census of art historians' archives. It is an integrated system that collects archival descriptions of notable art historians' collections and opens up unexplored research paths through the implementation of semantic web technologies. Being a nascent project, there is considerable margin for improvement in several aspects. At the moment, one of the most compelling needs is to better exploit the potential of ARTchives and, in particular, of Semantic Web technologies underlying the system. A noticeable way to achieve this goal is to enhance communication and visual aspects by introducing new data visualizations to the ones aready published in the dedicated section.
The work presented in this Notebook aims at providing further data visualization proposals, not yet published in the application but in the view of an actual implementation in the next releases of ARTchives. The starting point for the development of these visualizations were four fundamental research questions:
1. RQ1. What have been the places of education and activity of all the art historians recorded in ARTchives?
2. RQ2. What have been the places of education and activity of a particular art historian recorded in ARTchives?
3. RQ3. What were the relations of art historians with other experts of their times (other scholars, art collectors, connoisseurs etc.)?
4. RQ4. Which artists and personalities have been studied by art historians recorded in ARTchives?


## 1. The distribution of art historians’ places of education and activity: visualizing RQ1.

In [47]:
#! pip install rdflib
import rdflib
from rdflib import Namespace , Literal , URIRef
from rdflib.namespace import RDF , RDFS

# create an empty Graph
g = rdflib.ConjunctiveGraph()

# parse a local RDF file by specifying the format
result = g.parse("Desktop/dhdk_epds/resources/artchives.nq", format='nquads') #Desktop/dhdk_epds/resources/

# bind the uncommon namespaces
wd = Namespace("http://www.wikidata.org/entity/") # remember that a prefix matches a URI until the last slash (or hashtag #)
wdt = Namespace("http://www.wikidata.org/prop/direct/")
art = Namespace("https://w3id.org/artchives/")
rdfs = Namespace ("http://www.w3.org/2000/01/")

# Get the list of art historians in our graph "g"
arthistorians_list = set()
arthistorians_names = list()

# iterate over the triples in the graph
for s,p,o in g.triples(( None, wdt.P170, None)):
    for subj, prop, obj in g.triples((o, RDFS.label, None )):# people "o" are the creator "wdt.P170" of a collection "s"
        if "wikidata.org/entity/" in str(o):           # look for the substring to filter wikidata entities only
            arthistorians_list.add('<' + str(o) + '>')
            if obj.strip() not in arthistorians_names:
                arthistorians_names.append(obj.strip())
                arthistorians_names.append('<' + str(o) + '>')  # remember to transform them in strings! 
    
print(arthistorians_list)
print(arthistorians_names)


{'<http://www.wikidata.org/entity/Q18935222>', '<http://www.wikidata.org/entity/Q19997512>', '<http://www.wikidata.org/entity/Q6700132>', '<http://www.wikidata.org/entity/Q1271052>', '<http://www.wikidata.org/entity/Q1629748>', '<http://www.wikidata.org/entity/Q88907>', '<http://www.wikidata.org/entity/Q3051533>', '<http://www.wikidata.org/entity/Q61913691>', '<http://www.wikidata.org/entity/Q1641821>', '<http://www.wikidata.org/entity/Q85761254>', '<http://www.wikidata.org/entity/Q1715096>', '<http://www.wikidata.org/entity/Q1712683>', '<http://www.wikidata.org/entity/Q1089074>', '<http://www.wikidata.org/entity/Q90407>', '<http://www.wikidata.org/entity/Q2824734>', '<http://www.wikidata.org/entity/Q995470>', '<http://www.wikidata.org/entity/Q1296486>', '<http://www.wikidata.org/entity/Q3057287>', '<http://www.wikidata.org/entity/Q60185>', '<http://www.wikidata.org/entity/Q457739>', '<http://www.wikidata.org/entity/Q1373290>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.w

In [2]:
#!pip install SPARQLWrapper
from SPARQLWrapper import SPARQLWrapper, JSON
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"

# prepare the values to be queried
historians = ' '.join(arthistorians_list) # <uri1> <uri2> <uri3> ... <uriN>
#print(historians)
# prepare the query: for each historian in ARTchives check in wikidata if there are work or education places.
formationplace_query = """ 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT DISTINCT ?historian ?workplace ?workplace_label ?coordinates1 ?eduplace ?eduplace_label ?coordinates2 (group_concat(?type ; separator="; ") as ?place_type) (group_concat(?type_label ; separator="; ") as ?label)
WHERE {
    VALUES ?historian {"""+historians+"""} . 
    optional {?historian wdt:P108 ?workplace . 
    ?workplace rdfs:label ?workplace_label .
    FILTER (langMatches(lang(?workplace_label), "EN")) 
    ?workplace wdt:P625 ?coordinates1; wdt:P31 ?type . 
    ?type rdfs:label ?type_label . 
    FILTER (langMatches(lang(?type_label), "EN"))}
    optional {?historian wdt:P69 ?eduplace . 
    ?eduplace rdfs:label ?eduplace_label .
    FILTER (langMatches(lang(?eduplace_label), "EN")) 
    ?eduplace wdt:P625 ?coordinates2; wdt:P31 ?type . 
    ?type rdfs:label ?type_label . 
    FILTER (langMatches(lang(?type_label), "EN")) }
    } 
GROUP BY ?historian ?workplace ?workplace_label ?coordinates1 ?eduplace ?eduplace_label ?coordinates2 ?place_type ?label
    
"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(formationplace_query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

# manipulate the result
for result in results["results"]["bindings"]:
    #print(result)
    historian_uri = result["historian"]["value"]
    #print("historian:", historian_uri)
    if "workplace" in result: 
        workplace = result["workplace"]["value"]
        if "workplace_label" in result and "coordinates1" in result: 
            workplace_label = result["workplace_label"]["value"]
            work_coord = result["coordinates1"]["value"][6:-1].split(" ")
            #print(work_coord)
            #print("work:", workplace, workplace_label)

                    # only if uri, label and coords are found we add them to the graph
            g.add(( URIRef(historian_uri) , URIRef(wdt.P108) , URIRef(workplace) ))
            g.add(( URIRef(workplace) , RDFS.label , Literal(workplace_label) ))
            g.add(( URIRef(workplace) , URIRef(wdt.P625) , Literal(work_coord) ))
            if "place_type" in result: 
                type_label = result["label"]["value"].split(" ")[0]
                #print(type_label)
                place_type = result["place_type"]["value"].split(";")[0]
                #print("WORK ", workplace_label, type_label, place_type)
                if type_label != "":
                    g.add(( URIRef(workplace) , URIRef(wdt.P31) , URIRef(place_type) ))
                    g.add(( URIRef(place_type) , RDFS.label  , Literal(type_label) ))
                    
            
    if "eduplace" in result: 
        eduplace = result["eduplace"]["value"]
        #print(eduplace)
        if "eduplace_label" in result and "coordinates2" in result: 
            eduplace_label = result["eduplace_label"]["value"]
            eduplace_coord = result["coordinates2"]["value"][6:-1].split(" ")
            #print("education:", eduplace, eduplace_label)
                    # only if both uri and label are found we add them to the graph
            g.add(( URIRef(historian_uri) , URIRef(wdt.P69) , URIRef(eduplace) ))
            g.add(( URIRef(eduplace) , RDFS.label , Literal(eduplace_label) ))
            g.add(( URIRef(eduplace) , URIRef(wdt.P625) , Literal(eduplace_coord) ))
            if "place_type" in result: 
                type_label = result["label"]["value"].split(" ")[0]
                print(type_label)
                place_type = result["place_type"]["value"].split(";")[0]
                #print("EDUCATION ", eduplace_label, type_label, place_type)
                if type_label != "":
                    g.add(( URIRef(eduplace) , URIRef(wdt.P31) , URIRef(place_type) ))
                    g.add(( URIRef(place_type) , RDFS.label  , Literal(type_label) ))
            

private
private
private
private
college
public
private
private
public
open-access
private
public
private
private
private
private
private
private
public
private
private
private
private
private
private
private
private
public
public
public
university;
public
university;
university;
university;
public
public
private
public
private
private
public
private


In [3]:
g.serialize(destination='Desktop/dhdk_epds/resources/artchives2.nq', format='nquads') #Desktop/dhdk_epds/resources/
result = g.parse("Desktop/dhdk_epds/resources/artchives2.nq", format='nquads')



In [4]:
loc_list = ['country', 'city', 'village', 'capital', 'state', 'region', 'municipality', 'county', 'frazione', 'comune', 'city-state', 'enclave']
firstdict = {}
final = {}
for hist, prop, place in g.triples((None, wdt.P69, None)):
    for eduplace, hasname, eduplacename in g.triples((place, RDFS.label, None)):
        #print(eduplacename)
        for edup, hascoordinates, coord in g.triples((eduplace, wdt.P625, None)):
            for edupl, istype, placetype in g.triples((edup, wdt.P31, None)):
                for plctype, named, typelabel in g.triples((placetype, RDFS.label, None)):
                    #print(plctype)
                    for historian, p, name in g.triples(( hist, RDFS.label, None)):
                        if "wikidata.org/entity/" in str(historian):
                            type_label = typelabel
                            check =  any(item in loc_list for item in type_label)
                            if check:
                                loc = "geoloc"
                            else:
                                loc = 'institution'
                            key = (eduplacename.strip(), "<" + place.strip() +">", float(coord.split(" ")[1][1:-2][0:6]), float(coord.split(" ")[0][2:-2][0:6]), loc)
                            value = tuple(["<" + hist.strip() +">", name.strip()])
                            if key not in firstdict.keys():
                                firstdict[key] = set([value])
                            else:
                                firstdict[key].update([value])


                    
for hist, prop, place in g.triples((None, wdt.P108, None)):
    for place, hasname, workplacename in g.triples((place, RDFS.label, None)):
        for place, hasCoordinates, coord in g.triples((place, wdt.P625, None)):
            for edupl, istype, placetype in g.triples((place, wdt.P31, None)):
                for plctype, named, typelabel in g.triples((placetype, RDFS.label, None)):
                    for historian, p, name in g.triples(( hist, RDFS.label, None)):   
                        if "wikidata.org/entity/" in str(historian):
                            type_label = typelabel
                            check =  any(item in loc_list for item in type_label)
                            if check:
                                loc = "geoloc"
                            else:
                                loc = 'institution'
                            key = (workplacename.strip(), "<" + place.strip() +">", float(coord.split(" ")[1][1:-2][0:6]), float(coord.split(" ")[0][2:-2][0:6]), loc)
                            value = tuple(["<" + hist.strip() +">", name.strip()])
                            if key not in firstdict.keys():
                                firstdict[key] = set([value])
                            else:
                                firstdict[key].update([value])


for k, v in firstdict.items(): #creation of a dictionary that has as key a tuple with the info for a place and as values a list of tuples of art historians connected to that place.
    for el in v:
        if k not in final.keys():
            final[k] = [el]
        else:
            final[k].append(el)
            
for k, v in final.items():
    print(k,v)




('Harvard University', '<http://www.wikidata.org/entity/Q13371>', 42.374, -71.11, 'institution') [('<http://www.wikidata.org/entity/Q88907>', 'Ernst Kitzinger'), ('<http://www.wikidata.org/entity/Q457739>', 'Leo Steinberg'), ('<http://www.wikidata.org/entity/Q19997512>', 'Everett Fahy')]
('Humboldt University of Berlin', '<http://www.wikidata.org/entity/Q152087>', 52.518, 13.393, 'institution') [('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer')]
('Vassar College', '<http://www.wikidata.org/entity/Q2093794>', 41.686, -73.89, 'institution') [('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer'), ('<http://www.wikidata.org/entity/Q1296486>', 'Wolfgang Lotz')]
('New College', '<http://www.wikidata.org/entity/Q1376987>', 51.754, -1.251, 'institution') [('<http://www.wikidata.org/entity/Q3051533>', 'Ellis Waterhouse')]
('University of Virginia', '<http://www.wikidata.org/entity/Q213439>', 38.035, -78.5, 'institution') [('<http://www.wikidata.org/entity/Q19997512

In [5]:
related = {} #dictionary to collect the main subjects of an art historian -p921- present in ARTchives data 
for s,p,o in g.triples(( None, wdt.P170, None)):   
    if "wikidata.org/entity/" in str(o):           
        for hist, prop, obj in g.triples((o, wdt.P921, None)):     
            for subj, pr, name in g.triples(( hist, RDFS.label, None)):   
                key = tuple(['<' + str(hist) + '>', name.strip()])
                value = '<' + str(obj) + '>'
                if key not in related.keys(): #the dict has as keys the historians and as values the relative list of main subjects.
                    related[key] = [value]
                else:
                    related[key].append(value)
    
for k, v in related.items():
    print(k, v)



('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy') ['<http://www.wikidata.org/entity/Q37562>', '<http://www.wikidata.org/entity/Q37562>', '<http://www.wikidata.org/entity/Q37562>', '<http://www.wikidata.org/entity/Q213523>', '<http://www.wikidata.org/entity/Q213523>', '<http://www.wikidata.org/entity/Q213523>', '<http://www.wikidata.org/entity/Q2751>', '<http://www.wikidata.org/entity/Q2751>', '<http://www.wikidata.org/entity/Q2751>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q869690>', '<http://www.wikidata.org/entity/Q869690>', '<http://www.wikidata.org/entity/Q869690>', '<http://www.wikidata.org/entity/Q50641>', '<http://www.wikidata.org/entity/Q50641>', '<http://www.wikidata.org/entity/Q50641>', '<http://www.wikidata.org/entity/Q6373>', '<http://www.wikidata.org/entity/Q6373>', '<http://www.wikidata.org/entity/Q6373>', '<http://www.wikidata.org/

In [6]:
#import json

related_dict= {}


for k, v in related.items():  #for each art historian I extract those main subjects which are defined as places in wikidatata 
    relatedlist = ' '.join(v)
    

    query_results = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        SELECT DISTINCT ?place ?place_label ?coord (group_concat(?type_label ; separator="; ") as ?label) 
        WHERE {
            VALUES ?place {"""+relatedlist+"""} . 
            ?place rdfs:label ?place_label . 
            FILTER (langMatches(lang(?place_label), "EN")) . ?place wdt:P625 ?coord; wdt:P31 ?type . ?type rdfs:label ?type_label . FILTER (langMatches(lang(?type_label), "EN"))
            } 
            group by ?place ?place_label ?coord ?label
        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_results)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()
    
    #with open('q1.json', 'w') as f:
        #json.dump(results, f)

    for result in results["results"]["bindings"]:
        #print(v, result)
        place = '<' + str(result["place"]["value"]) + '>'
        place_name = result["place_label"]["value"]
        coord = result["coord"]["value"][6:-1].split(" ")
        type_label = result["label"]["value"].split("; ")[0]
        type_label_list = type_label.split(" ")
        check =  any(item in loc_list for item in type_label_list)
        if check:
            loc = "geoloc"
        else:
            loc = 'institution'
        key = tuple([place_name, place, float(coord[1][0:6]), float(coord[0][0:6]), loc])
        for value in v:
            if value == str(key[1]):
                #print(k[1], place_name)
                if key not in related_dict.keys():
                    related_dict[key] = set([k])
                else: 
                    related_dict[key].update([k])
                

                




            
for k, v in final.items():
    for el in v:
        if k not in related_dict.keys():
            related_dict[k] = set([el])
        else:
            related_dict[k].update([el])
    
    
for k, v in related_dict.items():
    print(k, v)





('Skulpturensammlung Dresden', '<http://www.wikidata.org/entity/Q869690>', 51.051, 13.744, 'institution') {('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy')}
('Siena', '<http://www.wikidata.org/entity/Q2751>', 43.318, 11.331, 'geoloc') {('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy')}
('British Museum', '<http://www.wikidata.org/entity/Q6373>', 51.519, -0.127, 'institution') {('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy')}
('New York University', '<http://www.wikidata.org/entity/Q49210>', 40.73, -73.99, 'institution') {('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer'), ('<http://www.wikidata.org/entity/Q19997512>', 'Everett Fahy'), ('<http://www.wikidata.org/entity/Q18935222>', 'Werner Cohn'), ('<http://www.wikidata.org/entity/Q457739>', 'Leo Steinberg'), ('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy'), ('<http://www.wikidata.org/entity/Q1712683>', 'Julius S. Held')}
('Italy', '<http://www.wikid

In [8]:
#!pip install ipywidgets
#!pip install ipyleaflet
from ipywidgets import HTML

from ipyleaflet import Map, Marker, Popup, LayersControl, AwesomeIcon

center = (41.080684, -30.683374)


m = Map(center=center, zoom=3, close_popup_on_click=False)



for k,v in related_dict.items():
    names = []
    for value in v:
        if value[0] not in names:
            names.append("<a href='http://artchives.fondazionezeri.unibo.it/historian-" + value[0][32:-1] + "'>" + value[1] + "</a>" + ". ")
    namelist = " ".join(names)
    #print(namelist)
    if 'geoloc' in k[4]:
        icon2 = AwesomeIcon(
        name = "map-marker",
        marker_color='blue',
        icon_color='white',
        spin=False
        )
        marker = Marker(icon = icon2, location=(k[2], k[3]))
        m.add_layer(marker)
        #message = HTML()
        #marker.popup = message
        #message.description = ""
    #message.value = "<b>" + k[0] + "</b>" + "<br>"  + namelist
    else:
        icon2 = AwesomeIcon(
        name = "bank",
        marker_color='green',
        icon_color='white',
        spin=False
            )
        marker = Marker(icon = icon2, location=(k[2], k[3]))
        m.add_layer(marker)
    message = HTML()
    marker.popup = message
    message.description = ""
    message.value = "<b>" + k[0] + "</b>" + "<br>" + namelist
    
        


m

Map(center=[41.080684, -30.683374], close_popup_on_click=False, controls=(ZoomControl(options=['position', 'zo…

## 2. The distribution of selected art historians’ places of education and activity: visualizing RQ2.

In [9]:
hist_dict = {} #dict that has as keys art historians and as values the list of related places. 
hist_list = []
for k, v in related_dict.items():
    for value in v:
        if value not in hist_list:
            hist_list.append(value)
    for el in hist_list:
        if el in v:
            if el not in hist_dict.keys():
                hist_dict[el] = [k]
            else:
                hist_dict[el].append(k)
        

for k, v in hist_dict.items():            
    print(k, v)

('<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy') [('Skulpturensammlung Dresden', '<http://www.wikidata.org/entity/Q869690>', 51.051, 13.744, 'institution'), ('Siena', '<http://www.wikidata.org/entity/Q2751>', 43.318, 11.331, 'geoloc'), ('British Museum', '<http://www.wikidata.org/entity/Q6373>', 51.519, -0.127, 'institution'), ('New York University', '<http://www.wikidata.org/entity/Q49210>', 40.73, -73.99, 'institution'), ('Italy', '<http://www.wikidata.org/entity/Q38>', 42.5, 12.5, 'geoloc'), ('University of Oxford', '<http://www.wikidata.org/entity/Q34433>', 51.755, -1.255, 'institution'), ('University of Cambridge', '<http://www.wikidata.org/entity/Q35794>', 52.205, 0.1172, 'institution')]
('<http://www.wikidata.org/entity/Q90407>', 'Richard Krautheimer') [('New York University', '<http://www.wikidata.org/entity/Q49210>', 40.73, -73.99, 'institution'), ('Rome', '<http://www.wikidata.org/entity/Q220>', 41.893, 12.482, 'geoloc'), ('Humboldt University of Berlin', '<

In [98]:
from ipyleaflet import LayerGroup
import random 
center = (41.080684, -30.683374)

colors = ['red', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'blue', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'gray', 'lightgray', 'black']


m = Map(center=center, zoom=3, close_popup_on_click=False)
control = LayersControl(position='topright')
m.add_control(control)

for k,v in hist_dict.items():
    layer_group = LayerGroup(layers=(), name=k[1]) #creating different layers for different historians
    m.add_layer(layer_group)
    #print(k, v)
    icon2 = AwesomeIcon(
        name = "map-marker",
        marker_color= random.choice(colors),
        icon_color='white',
        spin=False
        )
    for value in v:
        marker = Marker(icon = icon2, location=(value[2], value[3]))
        message = HTML()
        marker.popup = message
        message.description = ""
        message.value ="<b>" + "<a href='http://artchives.fondazionezeri.unibo.it/historian-" + k[0][32:-1] +"'>" + k[1] + "</a>" + "</b>" + "<br>"  + value[0]
        layer_group.add_layer(marker)





m

Map(center=[41.080684, -30.683374], close_popup_on_click=False, controls=(ZoomControl(options=['position', 'zo…

## 3. The distribution of art historians’ relations with experts of their times: visualizing RQ3.

In [13]:
people = []
for k, v in related.items():   
    relatedlist = ' '.join(v)
    #print(relatedlist)
    query_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?person ?person_label ?occupation ?deathdate
        WHERE {
            VALUES ?person {"""+relatedlist+"""} . 
            ?person wdt:P31 wd:Q5; rdfs:label ?person_label . 
            VALUES ?occupation {wd:Q1792450 wd:Q201788 wd:Q36180 wd:Q4164507 wd:Q1126160 wd:Q10732476} . 
            ?person wdt:P106 ?occupation ; wdt:P570 ?deathdate.
            FILTER (langMatches(lang(?person_label), "EN") ) 
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_res)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        #print(result)
        person = '<' + str(result["person"]["value"]) + '>'
        person_name = str(result["person_label"]["value"])
        death_date = int(result["deathdate"]["value"][0:4])
        key = tuple([person_name, person])
        if death_date > 1850:
            for value in v:
                if value == str(key[1]):
                    tupla = tuple([k[1], k[0], person_name, person, 2])
                    if tupla not in people:
                        people.append(tupla)

            
print(people)


[('John Pope-Hennessy', '<http://www.wikidata.org/entity/Q537874>', 'John Pope-Hennessy', '<http://www.wikidata.org/entity/Q537874>', 2), ('Luisa Vertova', '<http://www.wikidata.org/entity/Q61913691>', 'Bernard Berenson', '<http://www.wikidata.org/entity/Q359047>', 2), ('Werner Cohn', '<http://www.wikidata.org/entity/Q18935222>', 'Hans Jantzen', '<http://www.wikidata.org/entity/Q1580468>', 2), ('Ulrich Middeldorf', '<http://www.wikidata.org/entity/Q1715096>', 'Adolph Goldschmidt', '<http://www.wikidata.org/entity/Q70804>', 2), ('Ulrich Middeldorf', '<http://www.wikidata.org/entity/Q1715096>', 'Heinrich Wölfflin', '<http://www.wikidata.org/entity/Q123466>', 2), ('Ulrich Middeldorf', '<http://www.wikidata.org/entity/Q1715096>', 'Roberto Longhi', '<http://www.wikidata.org/entity/Q1361667>', 2), ('Kornél Fabriczy', '<http://www.wikidata.org/entity/Q995470>', 'Wilhelm von Bode', '<http://www.wikidata.org/entity/Q64607>', 2), ('Kornél Fabriczy', '<http://www.wikidata.org/entity/Q995470>', 'G

In [14]:
coll_related = {}
for coll,p,hist in g.triples(( None, wdt.P170, None)):
    for historian, proper, hist_name in g.triples((hist, RDFS.label, None)):     # people "o" are the creator "wdt.P170" of a collection "s"
        for collection, pr, coll_name in g.triples(( coll, RDFS.label, None)):   
            for col, prop, content in g.triples((coll, wdt.P921, None)):     
                for cont, pro, content_label in g.triples(( content, RDFS.label, None)): 
                    if "wikidata.org/entity/" in str(content):    
                        key = tuple([hist_name.strip(), '<' + str(hist) + '>'])
                        value = '<' + str(content) + '>'
                        if key not in coll_related.keys():
                            coll_related[key] = set([value])
                        else:
                            coll_related[key].add(value)
    


  
for k, v in coll_related.items():
    print( k, v)
    

#print(collrelatedlist)
 


('John Pope-Hennessy', '<http://www.wikidata.org/entity/Q537874>') {'<http://www.wikidata.org/entity/Q6373>', '<http://www.wikidata.org/entity/Q49210>', '<http://www.wikidata.org/entity/Q37562>', '<http://www.wikidata.org/entity/Q697096>', '<http://www.wikidata.org/entity/Q3305213>', '<http://www.wikidata.org/entity/Q49135>', '<http://www.wikidata.org/entity/Q4692>', '<http://www.wikidata.org/entity/Q5592>', '<http://www.wikidata.org/entity/Q7018>', '<http://www.wikidata.org/entity/Q7882150>', '<http://www.wikidata.org/entity/Q1332178>', '<http://www.wikidata.org/entity/Q5669>', '<http://www.wikidata.org/entity/Q1404472>', '<http://www.wikidata.org/entity/Q645717>', '<http://www.wikidata.org/entity/Q160236>', '<http://www.wikidata.org/entity/Q2607815>', '<http://www.wikidata.org/entity/Q213322>', '<http://www.wikidata.org/entity/Q860861>', '<http://www.wikidata.org/entity/Q5664>', '<http://www.wikidata.org/entity/Q537874>', '<http://www.wikidata.org/entity/Q5597>', '<http://www.wikidat

In [86]:
for k, v in coll_related.items(): 
    coll_list = ' '.join(v)
    query_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
        SELECT DISTINCT ?person ?person_label ?occupation ?deathdate
        WHERE {
            VALUES ?person {"""+coll_list+"""} . 
            ?person wdt:P31 wd:Q5; rdfs:label ?person_label ; wdt:P570 ?deathdate . 
            FILTER (?deathdate > "1850-01-01"^^xsd:dateTime) . 
            VALUES ?occupation {wd:Q1792450 wd:Q201788 wd:Q36180 wd:Q4164507 wd:Q1126160 wd:Q10732476} . 
            ?person wdt:P106 ?occupation .
            FILTER (langMatches(lang(?person_label), "EN")) . 
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_res)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        person = '<' + str(result["person"]["value"]) + '>'
        person_name = str(result["person_label"]["value"]) 
        death_date = int(result["deathdate"]["value"][0:4])
        key = tuple([person_name, person])
        print(death_date)
        for value in v:
            if value == str(key[1]):
                tupla = tuple([k[0], k[1], person_name, person, 3])
                if tupla not in people:
                    people.append(tupla)

#for el in people:           
    #rint(el)

1994
1949
1974
1949
1975
1978
1998
1998
1998
1998
1998
1998
1970
1997
1970
1970
1900
1900
1900
1955
1955
1955
1938
1897
1897
1961
1961
1961
1961
1961
1961
1966
1966
1966
2011
1917
1934
1913
1944
1994
1994
1994
1891
1891
1999
1961
1961
1933
1957
1997
1968
1968
2018
1968
1952
2018
1980
2011
2011
1954
1996
1996
1970
1970
1999
1985
1968
1968
1994
1994
1994
1968
1983
1983
1929
1929
1929
1929
1929
1929
1935
1983
1935
1937
1935
1929
1929
1929
1929
1929
1929
1971
1971
1990
1971
2003
2003
2003
2003
2003
2003
1968
1968
1968
1973
1973
2002
2002
1973
2012
1985
1985
1991
2004
2004
2018
2018
2019
2018
1959
1959
1959
1959
1959
1959
1970
1961
1947
1970
1970
1959
1959
1959
1959
1959
1959
1946
1978
2016
2016
2016
2016
2003
2003
2003
2003
2003
1961
2018
2018
2003
1968
1968
1968
1966
1944
1934
1953
1981
1960
1945
1945
1962
1990
1934
1957
2019
1998
1998
1998
1998
1998
1998
1994
1998
1998
1998
1998
1998
1998
2006
2001
2001
2001
2001
2001
2001
1996
1996
1993
1968
1968
1968


In [97]:
wdpeople_query = """ 
PREFIX wdt: <http://www.wikidata.org/prop/direct/>
SELECT DISTINCT ?historian ?student ?student_label ?master ?master_label ?influencer ?influencer_label ?signper ?signper_label ?birthdate ?deathdate 
WHERE {
        VALUES ?historian {"""+historians+"""} . 
        optional {?historian wdt:P802 ?student . 
        ?student rdfs:label ?student_label ; wdt:P570 ?deathdate.
        FILTER (langMatches(lang(?student_label), "EN"))
        FILTER (?deathdate > "1850-01-01"^^xsd:dateTime) . 
        }
        optional {?historian wdt:P1066 ?master . 
        ?master rdfs:label ?master_label ; wdt:P570 ?deathdate .
        FILTER (langMatches(lang(?master_label), "EN"))
        FILTER (?deathdate > "1850-01-01"^^xsd:dateTime) . 
        }
        optional {?historian wdt:P737 ?influencer . 
        ?influencer rdfs:label ?influencer_label ; wdt:P570 ?deathdate.
        FILTER (langMatches(lang(?influencer_label), "EN"))
        FILTER (?deathdate > "1850-01-01"^^xsd:dateTime) . 
        } 
        optional {?historian wdt:P3342 ?signper . 
        ?signper rdfs:label ?signper_label ; wdt:P570 ?deathdate.
        FILTER (langMatches(lang(?signper_label), "EN"))
        FILTER (?deathdate > "1850-01-01"^^xsd:dateTime) . 
        }
    } 
"""

sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
sparql_wd.setQuery(wdpeople_query)
    # set the returned format
sparql_wd.setReturnFormat(JSON)
    # get the results
results = sparql_wd.query().convert()

for result in results["results"]["bindings"]:
    #print(result)
    historian = result["historian"]["value"]
    if "influencer" in result: 
        influencer = result["influencer"]["value"]
        #print(influencer)
        if "influencer_label" in result: 
            influencer_label = result["influencer_label"]["value"]
            #print("influencer:", influencer, influencer_label)

                    # only if both uri and label are found we add them to 
            g.add(( URIRef(historian) , URIRef(wdt.P737) , URIRef(influencer) ))
            g.add(( URIRef(influencer) , RDFS.label , Literal(influencer_label) ))
            
           
    if "student" in result: 
        student = result["student"]["value"]
        print(historian)
        if "student_label" in result: 
            student_label = result["student_label"]["value"]
            print("historian:", historian, "student:", student, student_label)

                        # only if both uri and label are found we add them to 
            g.add(( URIRef(historian) , URIRef(wdt.P802) , URIRef(student) ))
            g.add(( URIRef(student) , RDFS.label , Literal(student_label) )) 
           
    
    if "master" in result: 
        master = result["master"]["value"]
        if "master_label" in result: 
            master_label = result["master_label"]["value"]
            #print("historian:", historian, "master:", master_label)

                        # only if both uri and label are found we add them to 
            g.add(( URIRef(historian) , URIRef(wdt.P1066) , URIRef(master) ))
            g.add(( URIRef(master) , RDFS.label , Literal(master_label) ))
            #if "deathdate" in result:
                #deathdate = str(result["deathdate"]["value"][0:4])
                #g.add(( URIRef(master) , URIRef(wdt.P570), Literal(deathdate) ))
                
    
    if "signper" in result: 
        signper = result["signper"]["value"]
        if "signper_label" in result: 
            signper_label = result["signper_label"]["value"]
            #print("signper:", signper, signper_label)

                        # only if both uri and label are found we add them to 
            g.add(( URIRef(historian) , URIRef(wdt.P3342) , URIRef(signper) ))
            g.add(( URIRef(signper) , RDFS.label , Literal(signper_label) ))
           
                

QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr= \nPREFIX wdt: <http://www.wikidata.org/prop/direct/>\nSELECT DISTINCT ?historian ?student ?student_label ?master ?master_label ?influencer ?influencer_label ?signper ?signper_label ?birthdate ?deathdate \nWHERE {\n    VALUES ?historian {<http://www.wikidata.org/entity/Q18935222> <http://www.wikidata.org/entity/Q19997512> <http://www.wikidata.org/entity/Q6700132> <http://www.wikidata.org/entity/Q1271052> <http://www.wikidata.org/entity/Q1629748> <http://www.wikidata.org/entity/Q88907> <http://www.wikidata.org/entity/Q3051533> <http://www.wikidata.org/entity/Q61913691> <http://www.wikidata.org/entity/Q1641821> <http://www.wikidata.org/entity/Q85761254> <http://www.wikidata.org/entity/Q1715096> <http://www.wikidata.org/entity/Q1712683> <http://www.wikidata.org/entity/Q90407> <http://www.wikidata.org/entity/Q1089074> <http://www.wikidat

In [88]:
g.serialize(destination='Desktop/dhdk_epds/resources/artchives3.nq', format='nquads') #Desktop/dhdk_epds/resources/
result = g.parse("Desktop/dhdk_epds/resources/artchives3.nq", format='nquads')



In [92]:
for hist, prop, influencer in g.triples((None, wdt.P737, None)):
    #print(hist)
    for influencer, hasname, influencername in g.triples((influencer, RDFS.label, None)):
        for historian, p, name in g.triples(( hist, RDFS.label, None)):
            for hi, isdead, deathdate in g.triples(( influencer, wdt.P570, None)):
                if "wikidata.org/entity/" in str(hist):
                    tuplawd = tuple([name.strip(), "<" + hist.strip() +">", influencername.strip(), "<" + influencer.strip() +">", 1])
                    if tuplawd not in people:
                        people.append(tuplawd)
                
for hist, prop, student in g.triples((None, wdt.P802, None)):
    #print(hist)
    for historian, p, name in g.triples(( hist, RDFS.label, None)):
        for student, hasname, studentname in g.triples((student, RDFS.label, None)):
            for hi, isdead, deathdate in g.triples(( student, wdt.P570, None)):
                if "wikidata.org/entity/" in str(hist):  
                    tuplawd = tuple([name.strip(), "<" + hist.strip() +">", studentname.strip(), "<" + student.strip() +">", 1])
                    if tuplawd not in people:
                        people.append(tuplawd)

for hist, prop, master in g.triples((None, wdt.P1066, None)):
    #print(hist)
    for historian, p, name in g.triples(( hist, RDFS.label, None)):
        for master, hasname, mastername in g.triples((master, RDFS.label, None)):
            for hi, isdead, deathdate in g.triples(( master, wdt.P570, None)):
                #print(deathdate)
                #death_date = int(deathdate)
                #if death_date > 1850:  
                if "wikidata.org/entity/" in str(hist):  
                    tuplawd = tuple([name.strip(), "<" + hist.strip() +">", mastername.strip(), "<" + master.strip() +">", 1])
                            #print(tuplawd)
                    if tuplawd not in people:
                        people.append(tuplawd)
                
                
for hist, prop, signper in g.triples((None, wdt.P3342, None)):
    #print(hist)
    for historian, p, name in g.triples(( hist, RDFS.label, None)):
        for signper, hasname, signpername in g.triples((signper, RDFS.label, None)):
            for hi, isdead, deathdate in g.triples(( signper, wdt.P570, None)):
                if "wikidata.org/entity/" in str(hist):  
                    tuplawd = tuple([name.strip(), "<" + hist.strip() +">", signpername.strip(), "<" + signper.strip() +">", 1])
                            #print(tuplawd)
                    if tuplawd not in people:
                        people.append(tuplawd)
                


In [93]:
import csv
with open('people.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['art_hist', 'art_hist_uri','person', 'person_uri', 'weight'])
    
    # access the list of tuples of the query results
    for res in people:
        # write in the csv
        my_writer.writerow([res[0], res[1], res[2], res[3], res[4]])

In [94]:
#! pip install pandas
import pandas as pd
# parse the csv into a dataframe
df = pd.read_csv("people.csv")
# print the first 5 rows
df.head()

,art_hist,art_hist_uri,person,person_uri,weight
0,John Pope-Hennessy,<http://www.wikidata.org/entity/Q537874>,John Pope-Hennessy,<http://www.wikidata.org/entity/Q537874>,2
1,Luisa Vertova,<http://www.wikidata.org/entity/Q61913691>,Bernard Berenson,<http://www.wikidata.org/entity/Q359047>,2
2,Werner Cohn,<http://www.wikidata.org/entity/Q18935222>,Hans Jantzen,<http://www.wikidata.org/entity/Q1580468>,2
3,Ulrich Middeldorf,<http://www.wikidata.org/entity/Q1715096>,Adolph Goldschmidt,<http://www.wikidata.org/entity/Q70804>,2
4,Ulrich Middeldorf,<http://www.wikidata.org/entity/Q1715096>,Heinrich Wölfflin,<http://www.wikidata.org/entity/Q123466>,2


In [95]:
#!pip install pyvis

from pyvis import network as net



people_net = net.Network(height="750px", width="100%", bgcolor="white", font_color="#23f5ad", notebook="True", heading="The distribution of art historians' relations with experts of their times")

# set the physics layout of the network
people_net.barnes_hut()
people_data = pd.read_csv("people.csv")

sources = people_data['art_hist']
targets = people_data['person']
weights = people_data['weight']
uri = people_data['art_hist_uri']

edge_data = zip(sources, targets, weights)

for e in edge_data:
    #print(e)
    src = e[0]
    dst = e[1]
    w = e[2]


    people_net.add_node(src, src, title=src, color= "#1cae81", shape='dot')
    people_net.add_node(dst, dst,  title=dst, color= "#1cae81", shape='dot')
    if w == 1:
        people_net.add_edge(src, dst, value=w, color="orange")
    elif w == 2:
        people_net.add_edge(src, dst, value=w, color="grey")
    elif w == 3:
        people_net.add_edge(src, dst, value=w, color="#1cae81")

neighbor_map = people_net.get_adj_list()

# add neighbor data to node hover data
for node in people_net.nodes:
    #print(node)
    node["title"] = "<b>" + node["title"] + "</b>"+ " relations:<br>" + "<br>".join(neighbor_map[node["id"]])
    node["size"] = 70 + (len(neighbor_map[node["id"]])*10)  #len(neighbor_map[node["id"]]) #70 + (len(neighbor_map[node["id"]])*10) 
    node["label"] = node["id"] 
    node["borderWidthSelected"] = 5 
    if node["label"] in arthistorians_names:
        node["color"] = "#23f5ad"
        uripos = arthistorians_names.index(node["label"])+1
        uri = arthistorians_names[uripos]
        #print(node["label"], uri)
        node["title"] = "<br>" + "<b>" + "<a href='http://artchives.fondazionezeri.unibo.it/historian-" + uri[32:-1] + "'>" + node["label"] + "</a>" +  "</b>" +  "<i>" + "  relations:<br>" + "</i>"+ "<hr>" + "<br>".join(neighbor_map[node["id"]]) 
    else: 
        node["title"] = "<br>" + "<b>" + node["label"] + "</b>" +  "<i>" + "  relations:<br>" + "</i>"+ "<hr>"+ "<br>".join(neighbor_map[node["id"]]) + "<br>" 
        


people_net.show("people.html")


## 4. The distribution of art historians’ relations with their subjects of study: visualizing RQ4.

In [28]:

# sparql

query_periods = g.query(
    """PREFIX wdt: <http://www.wikidata.org/prop/direct/>
    SELECT ?hist_label ?hist ?period (SAMPLE(?label) AS ?period_label) 
    WHERE {?coll <https://w3id.org/artchives/hasSubjectPeriod> ?period ; rdfs:label ?coll_label 
    . ?coll wdt:P170 ?hist . ?hist rdfs:label ?hist_label .
    ?period rdfs:label ?label . 
    }
    GROUP BY ?period ?label ?hist ?hist_label 
    ORDER BY ?period""")

periods = set()
period_dict = {}
for result in query_periods:
    #print(result)
    hist = result[0].strip()
    period = result[2].strip()
    #print(period)
    if hist not in period_dict.keys():
        period_dict[hist] = set([period])
    else:
        period_dict[hist].add(period)
    

for k,v in period_dict.items():
    for value in v:
        periods.add('<' + str(value) + '>') 
#print(periods)

In [29]:
period_list = ' '.join(periods)
period_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?period ?period_label ?startdate ?enddate
        WHERE {
            VALUES ?period {"""+period_list+"""} . 
            ?period wdt:P580 ?startdate ; wdt:P582 ?enddate; rdfs:label ?period_label .
            FILTER (langMatches(lang(?period_label), "EN"))
            
            } 
        """
sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
sparql_wd.setQuery(period_res)
    # set the returned format
sparql_wd.setReturnFormat(JSON)
    # get the results
results = sparql_wd.query().convert()

period_dict = {}
for result in results["results"]["bindings"]:
    period = '<' + str(result["period"]["value"]) + '>'
    period_label = result["period_label"]["value"]
    start = int(result["startdate"]["value"][0:4])
    end = int(result["enddate"]["value"][0:4])
    key = tuple([period, period_label])
    years = tuple([start, end])
    if key not in period_dict.keys():
        period_dict[key] = years

#for k, v in period_dict.items():
    #print(k,v)


In [30]:
artist_rel = []
artist_dict = {}
artist_set = set()

for k, v in coll_related.items(): 
    coll_list = ' '.join(v)
    query_res = """
        PREFIX wdt: <http://www.wikidata.org/prop/direct/>
        PREFIX wd: <http://www.wikidata.org/entity/>
        SELECT DISTINCT ?person ?person_label ?occupation ?birthdate ?deathdate
        WHERE {
            VALUES ?person {"""+coll_list+"""} . 
            ?person wdt:P31 wd:Q5; rdfs:label ?person_label .
            VALUES ?occupation {wd:Q1281618 wd:Q42973 wd:Q483501 wd:Q1028181 wd:Q329439} . 
            ?person wdt:P106 ?occupation ; wdt:P569 ?birthdate ; wdt:P570 ?deathdate.
            FILTER (langMatches(lang(?person_label), "EN")) . 
            } 

        """

    # set the endpoint 
    sparql_wd = SPARQLWrapper(wikidata_endpoint)
    # set the query
    sparql_wd.setQuery(query_res)
    # set the returned format
    sparql_wd.setReturnFormat(JSON)
    # get the results
    results = sparql_wd.query().convert()

    for result in results["results"]["bindings"]:
        #print(result)
        artist = '<' + str(result["person"]["value"]) + '>'
        artist_name = str(result["person_label"]["value"]) 
        death = str(result["deathdate"]["value"]) 
        birth = str(result["birthdate"]["value"])
        key = tuple([artist, artist_name, int(birth[0:4]), int(death[0:4])])
        for value in v:
            #print(value)
            if value == artist:
                if key not in artist_dict.keys():      
                    artist_dict[key] = set([k[0]])
                else:
                    artist_dict[key].add(k[0])
                artist_set.add(artist)
                
#for k, v in artist_dict.items():
    #print(k, v)
    

In [31]:
for k, v in artist_dict.items():
    for value in v:
        tupla = tuple([k[1], value, 1])
        if tupla not in artist_rel:
            artist_rel.append(tupla)
        for el in v:
            if value != el:
                tupla = tuple([el, value, 1])
                tuplabis = tuple([value, el, 1])
                if tuplabis not in artist_rel:
                    artist_rel.append(tupla)
    
#print(artist_rel)

In [32]:
from collections import Counter

count_dict = Counter(artist_rel)
artist_rel_final = []

for k, v in count_dict.items():
    if v != 1:
        x = list(k)
        x[2] = v
        k = tuple(x)
    artist_rel_final.append(k)
    
#print(artist_rel_final)

In [33]:
for k, v in coll_related.items():
    #print(k, v)
    for key, value in period_dict.items():
        for artist, hist in artist_dict.items():
            if key[0] in v and artist[0] in v:
                if (value[0] <= artist[2] and value[1] >= artist[3]) or (value[0] >= artist[2] and value[1] <= artist[3]) or (value[0] >= artist[2] and (value[1] + 15) >= artist[3] and artist[2] >= value[1]) or (value[0] <= artist[2] and (value[1] - 15) >= artist[2] and value[1] <= artist[3]):
                    tupla = tuple([k[0], artist[1], 2])
                    #print(tupla)
                    tupla_remove1 = tuple([k[0], artist[1], 1])
                    tupla_remove2 = tuple([artist[1], k[0], 1])
                    if tupla_remove1 in artist_rel_final: 
                        artist_rel_final.remove(tupla_remove1)
                    elif tupla_remove2 in artist_rel_final: 
                        artist_rel_final.remove(tupla_remove2)
                    artist_rel_final.append(tupla)
                        
print(artist_rel_final)                 

[('Kornél Fabriczy', 'John Pope-Hennessy', 1), ('Ernst Steinmann', 'John Pope-Hennessy', 2), ('Ernst Steinmann', 'Kornél Fabriczy', 1), ('Everett Fahy', 'John Pope-Hennessy', 1), ('Ernst Steinmann', 'Everett Fahy', 1), ('Donatello', 'John Pope-Hennessy', 1), ('Federico Zeri', 'John Pope-Hennessy', 1), ('Cecil Beaton', 'John Pope-Hennessy', 1), ('Gustav Ludwig', 'Leo Steinberg', 1), ('Fritz Heinemann', 'Leo Steinberg', 1), ('Luisa Vertova', 'Leo Steinberg', 1), ('Titian', 'Gustav Ludwig', 1), ('Fritz Heinemann', 'Gustav Ludwig', 3), ('Luisa Vertova', 'Gustav Ludwig', 1), ('Luisa Vertova', 'Fritz Heinemann', 1), ('Carlo Carrà', 'Werner Cohn', 1), ('Richard Krautheimer', 'Werner Cohn', 1), ('Lorenzo Ghiberti', 'Richard Krautheimer', 1), ('Stefano Bardini', 'Werner Cohn', 1), ('El Greco', 'Ellis Waterhouse', 1), ('Giorgione', 'Ellis Waterhouse', 1), ('Filippo Napoletano', 'Luigi Salerno', 1), ('Giovanni Lanfranco', 'Luigi Salerno', 1), ('Luigi Salerno', 'Federico Zeri', 1), ('Caravaggio', 

In [34]:
with open('artists.csv', mode='w') as my_file:
    my_writer = csv.writer(my_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    # write the column names
    my_writer.writerow(['subj1','subj2','weight'])
    
    # access the list of tuples of the query results
    for res in artist_rel_final:
        # write in the csv
        my_writer.writerow([res[0], res[1], res[2]])

In [35]:
# parse the csv into a dataframe
df = pd.read_csv("artists.csv")
# print the first 5 rows
df.head()

,subj1,subj2,weight
0,Kornél Fabriczy,John Pope-Hennessy,1
1,Ernst Steinmann,John Pope-Hennessy,2
2,Ernst Steinmann,Kornél Fabriczy,1
3,Everett Fahy,John Pope-Hennessy,1
4,Ernst Steinmann,Everett Fahy,1


In [80]:
people_net = net.Network(height="750px", width="100%", bgcolor="white", font_color="#1cae81", notebook="True", heading="The distribution of art historians' relations with their subjects of study")

# set the physics layout of the network
people_net.barnes_hut()
people_data = pd.read_csv("artists.csv")

sources = people_data['subj1']
targets = people_data['subj2']
weights = people_data['weight']


edge_data = zip(sources, targets, weights)

for e in edge_data:
    #print(e)
    src = e[0]
    dst = e[1]
    w = e[2]


    people_net.add_node(src, src, title=src, color= "#1cae81")
    people_net.add_node(dst, dst, title=dst, color= "#1cae81")
    if w == 1:
        people_net.add_edge(src, dst, value=w, color="grey")
    elif w == 2:
        people_net.add_edge(src, dst, value=w, color="#1cae81")
    elif w == 3:
        people_net.add_edge(src, dst, value=w, color="black")
    
        

neighbor_map = people_net.get_adj_list()

# add neighbor data to node hover data
for node in people_net.nodes:
    #print(node)
    
    node["size"] = 80 + (len(neighbor_map[node["id"]])*10)   
    node["borderWidthSelected"] = 5
    node["label"] = node["id"] 
    if node["label"] in arthistorians_names:
        node["color"] = "#23f5ad"
        uripos = arthistorians_names.index(node["label"])+1
        uri = arthistorians_names[uripos]
        #print(node["label"], uri)
        node["title"] = "<br>" + "<b>" + "<a href='http://artchives.fondazionezeri.unibo.it/historian-" + uri[32:-1] + "'>" + node["label"] + "</a>" +  "</b>" +  "<i>" + "  relations:<br>" + "</i>"+ "<hr>" + "<br>".join(neighbor_map[node["id"]]) 
    else: 
        node["title"] = "<br>" + "<b>" + node["label"] + "</b>" +  "<i>" + "  relations:<br>" + "</i>"+ "<hr>"+ "<br>".join(neighbor_map[node["id"]]) + "<br>" 
        
people_net.show("people.html")